# Missing Data


Often data sources are incomplete, which means you will have missing data, you have 3 basic options for filling in missing data (you will personally have to make the decision for what is the right approach:

Just keep the missing data points.

Drop them missing data points (including the entire row)

Fill them in with some other value.

Let's cover examples of each of these methods!


In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('missingdata').getOrCreate()


In [2]:
df = spark.read.csv("ContainsNull.csv", header=True, inferSchema=True)

In [3]:
df.show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



Drop the missing data
You can use the .na functions for missing data. The drop command has the following parameters:
df.na.drop(how='any', thresh=None, subset=None)

* param how: 'any' or 'all'.

    If 'any', drop a row if it contains any nulls.
    If 'all', drop a row only if all its values are null.

* param thresh: int, default None

    If specified, drop rows that have less than `thresh` non-null values.
    This overwrites the `how` parameter.

* param subset: 
    optional list of column names to consider.

In [6]:
#Has to have at elase 2 Non null values
df.na.drop().show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp4|Cindy|456.0|
+----+-----+-----+



In [8]:
df.na.drop(subset=['Sales']).show()


+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [9]:
df.na.drop(how='any').show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp4|Cindy|456.0|
+----+-----+-----+



In [10]:
df.na.drop(how='all').show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



# 